## 3.3 Concise Implementation of Linear Regression
### 3.3.1 Generating the Dataset

In [1]:
from d2l import mxnet as d2l
from mxnet import autograd,gluon, np, npx
npx.set_np()

In [2]:
true_w=np.array([2,-3.4])
true_b=4.2
features, labels=d2l.synthetic_data(true_w,true_b,1000)

### 3.3.2 Reading the Dataset

In [3]:
def load_array(data_arrays, batch_size, is_train=True):
    """Construct a gluon data iterator"""
    dataset=gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

In [4]:
batch_size=10
data_iter=load_array((features, labels), batch_size)

In [5]:
next(iter(data_iter))

[array([[-0.42182946, -0.22930646],
        [ 0.8301537 ,  0.77279943],
        [ 1.3005846 , -0.6116391 ],
        [-0.35396403,  0.00955428],
        [ 1.3111951 ,  1.2040559 ],
        [-0.9185634 ,  2.0088325 ],
        [ 0.8166484 , -0.39319617],
        [-0.6441541 ,  0.7260685 ],
        [-0.28378814,  0.6376775 ],
        [-0.694951  , -0.00414529]]),
 array([[ 4.1577287 ],
        [ 3.225766  ],
        [ 8.882317  ],
        [ 3.4600995 ],
        [ 2.7131693 ],
        [-4.4819984 ],
        [ 7.173542  ],
        [ 0.44366717],
        [ 1.4705384 ],
        [ 2.8267581 ]])]

### 3.3.3 Defining the Model

In [6]:
from mxnet.gluon import nn
net=nn.Sequential()
net.add(nn.Dense(1))

### 3.3.4 Initializing Model Parameters

In [8]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))

### 3.3.5 Defining the Loss Function

In [9]:
loss=gluon.loss.L2Loss()

### 3.3.6 Defining the Optimization Algorithm

In [10]:
from mxnet import gluon
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.03})

In [11]:
num_epochs=3
for epoch in range(num_epochs):
    for X,y in data_iter:
        with autograd.record():
            l=loss(net(X),y)
        l.backward()
        trainer.step(batch_size)
    l=loss(net(features), labels)
    print(f"epoch: {epoch+1}, loss: {l.mean().asnumpy():f}")

epoch: 1, loss: 0.050306
epoch: 2, loss: 0.000215
epoch: 3, loss: 0.000052


In [12]:
w=net[0].weight.data()
print(f"error in estimating w: {true_w-w.reshape(true_w.shape)}")
b=net[0].bias.data()
print(f"error in estimating b: {true_b-b}")

error in estimating w: [ 5.9127808e-04 -4.6253204e-05]
error in estimating b: [0.00153303]
